# Cleaning and Feature Engineering of the Emory RCC Dataset

## Steps to Take
1. Load dataset into pandas dataframe
2. Drop unnecessary or 100% empty columns
3. Parse dates 
4. Decouple compounded features
5. Transform data
6. Drop single value columns
7. Clean dataframe dtypes
8. Handle outliers
9. Imputation
10. Feature selection
11. Sample dropping
12. Scale and Normalize
13. Clean col names
14. Save cleaned CSV

## Imports

In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

2024-02-22 12:25:23.620633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 12:25:24.734264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 1. Load dataset into pandas dataframe

In [39]:
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y')
df = pd.read_csv('../Data Sources/DO NOT MODIFY/US_RCC_Database.csv', parse_dates=True, date_format='%m/%d/%Y')
# Drop all rows and columns that are completely empty
empty_cols = [col for col in df.columns if df[col].isnull().all()]
empty_rows = [row for row in df.index if df.iloc[row].isnull().all()]
print(empty_cols)
print(empty_rows)
df.drop(labels=empty_cols, axis=1, inplace=True)
df.drop(index=empty_rows, axis=0, inplace=True)
data_shape = df.shape
print(data_shape)

['Patient ID', 'Patient MRN', 'Last Name', 'First Name', 'Patient SSN', 'Date of Birth', 'Fistula ']
[]
(1881, 408)


Having run a simple df.shape, this dataset has a total of 415 features and 1881 samples. That is a lot a features for the model to draw from, so we need to make sure they are all relevant to the task.

There are a few columns with duplicate names, we need to go through and adjust these to be more descriptive.

In [45]:
# Note: First result is 'Primary v. Recurrence', not relevant
dup_cols = df.columns[df.columns.str.contains("\.")].to_list()[1:]
new_names = ['Date of Initiation_NeoadjChemoRad',
            'Date of Completion_NeoadjChemoRad',
            'Number of Mesorectal Nodes by MRI_Post',
            'Number of Mesorectal Nodes by ERUS_Post',
            'Number of Retroperitoneal Nodes by CT_Post',
            'Number of Retroperitoneal Nodes by MRI_Post',
            'Number of Retroperitoneal Nodes by PET-CT_Post',
            'Number of Pelvic Nodes on CT_Post',
            'Number of Pelvic Nodes on MRI_Post',
            'Number of Pelvic Nodes on PET-CT_Post',
            'Involvement of Pelvic Sidewall_Post',
            'Distal Circumferential or Radial Margin (mm)_Post',
            'Sphincter Involvement_Post',
            'Invasion into Reproductive Organs_Post',
            'Invasion into Bladder_Post',
            'Invasion into Sacrum_Post',
            'Invasion of Sacral Nerve Roots_Post',
            'Involvement of Pelvic Sidewall_Op',
            'Omental Flap to Pelvis_Op_APR',
            'T-Stage_Recurrence',
            'N-Stage_Recurrence',
            'Type of Operation of Rectal Tumor_Recurrence',
            'Distal Margin Distance (cm)_Recurrence',
            'Radial Margin Distance (mm)_Recurrence',
            'Date of Diagnosis_PostOp_Leak',
            'Acute Renal Failure_PostOp',
            'Date of Discharge_Readmission',
            'Date of Initiation_AdjChemo',
            'Date of Completion_AdjChemo',
            'Date of Initiation_AdjChemoRad',
            'Date of Completion_AdjChemoRad',
            'Radiation Technique_AdjChemoRad',
            'Date of Initiation_AdjRad',
            'Date of Completion_AdjRad',
            'Sacrum_Recurrence_Locoregional',
            'Bladder_Recurrence_Locoregional',
            'Seminal Vesicles_Recurrence_Locoregional',
            'Prostate_Recurrence_Locoregional',
            'Vagina_Recurrence_Locoregional',
            'Ureter_Recurrence_Locoregional',
            'Grey (Gy)_Recurrence_Rad',
            'Ablation_Recurrence']

df.rename(columns=dict(zip(dup_cols, new_names)), inplace=True)
assert df.shape == data_shape

## 2. Drop unnecessary or empty columns
First, to aid in the cleaning process, we'll extract the number of missing and unique (if discrete) values for each feature and save to a file for future reference

In [46]:
print("# categorical features: ", len(df.select_dtypes(include=['object']).columns.tolist()))
print("# unique values by categorical features: ")
print("% missing values by column: \n", df.isnull().sum().div(data_shape[0]).sort_values(ascending=False))

# categorical features:  346
# unique values by categorical features: 
% missing values by column: 
 Radiotherapy Complications                 0.999468
Complication after Stenting                0.999468
Intervention for Peritoneal Perforation    0.997873
Radiation Technique_AdjChemoRad            0.996810
Necrosis (%)                               0.996279
                                             ...   
Previous Diagnosis of Prostate Cancer      0.000000
Previous Diagnosis of GYN Cancer           0.000000
Disseminated Cancer                        0.000000
Prior PELVIC Radiation                     0.000000
Database ID                                0.000000
Length: 408, dtype: float64


### Unnecessary columns (a * indicates that the column is a duplicate name and needs to be renamed):
- Watch and Wait Protocol
- Adjuvant Radiotherapy
- Grey (Gy)
- Radiation Technique
- Date of Initiation-Radiotherapy *
- Date of Completion-Radiotherapy *
- Radiotherapy Complications

In [12]:
empty_cols = df.columns[df.isnull().mean() < 1]
df = df[empty_cols]
print(empty_cols)

Index(['Database ID', 'Gender', 'Age', 'Race', 'Height (cm)', 'Weight (kg)',
       'BMI', 'Zip Code', 'Health Insurance', 'ASA Class',
       ...
       'Ovary', 'Bone', 'Distant LNs', 'Chemotherapy', 'Chemotherapy Regimen',
       'Radiation', 'Grey (Gy)_Recurrence_Rad', 'Ablation_Recurrence',
       'Surgery', 'COMMENTS'],
      dtype='object', length=408)


In [ ]:
drop_cols = ['Watch and Wait Protocol', 'Adjuvant Radiotherapy', 'Grey (Gy)', 'Radiation Technique', 'Date of Initiation-Radiotherapy', 'Date of Completion-Radiotherapy', 'Radiotherapy Complications']
df = df.drop(columns=drop_cols)
df

## 3. Parse dates
In this example, I will be selecting each column by hand that contains a date format. This is probably not the best way to do this, but I would like to conserve the naming conventions of the dataset and I'm too lazy to figure out how to do this automatically via Pandas. Instead of parsing dates into day, month, and year columns, each column will be replaced with the # of days since the date of diagnosis. This is done to avoid the model drawing inferences between past time frames and overfitting to the training dataset. This also requires that 'Date of Diagnosis' be dropped.
### It is probably much better to use pd.to_timedelta for this, I didn't notice it existed until later
### Colmuns with date data (a * indicates that the column is a duplicate name and needs to be renamed):
- Date of Diagnosis
- Date of First Oncologic Consultation
- Date of Biopsy
- Date of Initiation-Neoadjuvant Chemotherapy *
- Date of Completion-Neoadjuvant Chemotherapy *
- Date of Initiation-Neoadjuvant Chemoradiation *
- Date of Completion-Neoadjuvant Chemoradiation *
- Date of Surgery
- Date of Diagnosis-Anastomic Leak *
- Date of Intervention-Anastomic Leak *
- Date of Drain Removal
- Date of Ileostomy Reversal 
- Date of Reoperation
- Date of ICU Admission
- Date of ICU Discharge
- Date of Discharge
- Date of Readmission
- Date of Discharge-Readmission *
- Date of Initiation-Adjuvant Chemotherapy *
- Date of Completion-Adjuvant Chemotherapy *
- Date of Initiation-Adjuvant Chemoradiation *
- Date of Completion-Adjuvant Chemoradiation *
- Date of Last Follow-up/Death
- Date of Death
- Date of Recurrence
- Date Rectal Stent Placed

### Handling dates BEFORE date of diagnosis
In this dataset, there are features where the date values precede the date of diagnosis. This causes issues when we use the date of diagnosis as the 'origin' date as the distance between the two could be negative, the current solution is to just let the values be negitive. If this could be attributed to a loss in accuracy, this solution could be replaced with simply dropping dates occurring before the origin or changing the origin date.

In [ ]:
from datetime import date
date_cols = ['Date of First Oncologic Consultation', 'Date of Biopsy', 'Date of Initiation-Neoadjuvant Chemotherapy', 'Date of Initiation-Neoadjuvant Chemoradiation', 'Date of Surgery', 'Date of Diagnosis-Anastomic Leak', 'Date of Intervention-Anastomic Leak', 'Date of Reoperation', 'Date of ICU Admission', 'Date of Initiation-Adjuvant Chemotherapy', 'Date of Initiation-Adjuvant Chemoradiation', 'Date of Last Follow-up/Death', 'Date of Recurrence', 'Date Rectal Stent Placed']
diagnosis_dates = df['Date of Diagnosis']
for col in date_cols:
    values = df[col]
    null_values = values.isnull()
    index = df.columns.get_loc(col)
    col_tokens = col.split()
    new_name = 'Days to'
    for token in col_tokens[2:]:
        new_name = new_name + ' ' + token
    new_values = []
    count = 0
    null_vals = values.isnull()
    for value in values:
        if null_vals.get(count) == False:
            diagnosis = diagnosis_dates.get(count).split('/')
            date_to = value.split('/')
            d1 = date(int(date_to[2]), int(date_to[0]), int(date_to[1]))
            d2 = date(int(diagnosis[2]), int(diagnosis[0]), int(diagnosis[1]))
            new_values.append((d1 - d2).days)
        else:
            new_values.append(None)
        count += 1
    df.insert(index, new_name, new_values, allow_duplicates=False)
df

In [ ]:
print(date_cols)
df = df.drop(date_cols, axis = 1)
df

In [ ]:
drop_dates = ['Date of Diagnosis', 'Date of Completion-Neoadjuvant Chemotherapy', 'Date of Completion-Neoadjuvant Chemoradiation', 'Date of Drain Removal', 'Date of Ileostomy Reversal', 'Date of ICU Discharge', 'Date of Discharge', 'Date of Readmission', 'Date of Discharge-Readmission', 'Date of Completion-Adjuvant Chemotherapy', 'Date of Completion-Adjuvant Chemoradiation', 'Date of Death']
df = df.drop(columns=drop_dates)
df

## 4. Decouple compounded features
For example, a list of complications from a treatment. These should be split into multiple columns and one-hot encoded.
### Columns to be parsed (a * indicates that the column is a duplicate name and needs to be renamed): 
- Complication During Neoadjuvant Treatment (Side Note: This should be categorical, 'Yes' values should be removed and Imputated with median or similar)
- Type of Intraoperative Complication 
- Organs Invaded
- Neoadjuvant Chemo Regimen (Side Note: This feature needs to be adaquitely cleaned, some of the values are not consistent or straightforward)
- Adjuvant Chemo Regimen (Side Note: This feature needs to be adaquitely cleaned, some of the values are not consistent or straightforward)
- Chemotherapy Regimen-Recurrence (Side Note: This feature needs to be adaquitely cleaned, some of the values are not consistent or straightforward) *
### Processes
With many unique values, binning/n-grams will need to be practiced to handle misspellings or other mistakes. This will be done by splitting each value into tokens and compliling a list of unique values to inspect and bin. Many of these features also have secondary columns that indicate whether or not the specific features are present. For example, a Chemotherapy field with yes or no values that correspond to the entries in the Chemotherapy Regimen-Recurrence column. These will need to be removed to avoid confusing the model.

Diagnosis at readmission was discovered to need binning during dtype cleaning. Since binning is very time intensive with a dataset like this and the Nan count in the feature is 1358 after dtype cleaning pass 1, we will bin this iff it makes it past feature selection.

In [ ]:
count = df['Diagnosis at Readmission'].isna().sum()
print(count)

enumerator = 0
unique_tokens = []
for entry in therapy_complications:
    if null_values.get(enumerator) == False:
        if entry == 'No':
            new_values.append('No')
        elif entry == 'Yes':
            new_values.append("Yes")
        else:
            new_values.append('Yes')
        entry = entry.replace(';', ',')
        entry = entry.replace(', and', ',')
        tokens = entry.split(', ')
        trigger = False
        for token in tokens:
            token = token.lower()
            for u in unique_tokens:
                if token == u.lower():
                    trigger = True
            if (not trigger) and (token != 'no') and (token != 'yes') and (token != 'per notes'):
                unique_tokens.append(token)
            trigger = False
    else:
        new_values.append('No')
    enumerator += 1

In [ ]:
def get_unique_tokens(col_name, split_indicators, df):
    index = df.columns.get_loc(col_name)
    values = df[col_name]
    null_values = values.isnull()
    enumerator = 0
    unique_tokens = []
    for value in values:
        if null_values.get(enumerator) == False:
            for indicator in split_indicators:
                value = value.replace(indicator, ',')
            tokens = value.split(',')
            trigger = False
            for token in tokens:
                token = token.lower()
                for u in unique_tokens:
                    if token == u.lower():
                        trigger = True
                if (not trigger):
                    unique_tokens.append(token)
                trigger = False
        enumerator += 1
    return unique_tokens

### Binning method
Each column to be binned has its own dict with the keys being the bins to place the values and the items being an array of unique values that belong in that bin. Chemotherapies will be one hot encoded and given a 'swapped to' column containing the secondary regimen bin to let the model know that it had to be changed. All other features will have their unique values concatenated into their respective bin so the model can draw inferences between the magnitude of bins.

In [ ]:
def bin(col_name, bin_dict, df, chemo_indicator = False):
    col_data = df[col_name]
    null_entries = col_data.isnull()
    col_index = df.columns.get_loc(col_name)
    chemo_indicies = []
    chemo_secondary = []
    key_num = 0
    for key in bin_dict:
        new_values = []
        items = bin_dict[key]
        iterator = 0
        for data in col_data:
            if (key_num == 0) and chemo_indicator:
                chemo_indicies.append(None)
                chemo_secondary.append('None')
            new_values.append(0)
            if null_entries.get(iterator) == False:
                for item in items:
                    if item in data.lower():
                        if chemo_indicator:
                            therapy_index = data.lower().find(item)
                            if chemo_indicies[iterator] == None:
                                chemo_indicies[iterator] = therapy_index
                            elif chemo_indicies[iterator] > therapy_index:
                                chemo_secondary[iterator] = key
                                chemo_indicies[iterator] = therapy_index
                            new_values[iterator] = 1
                        else:
                            scale = 1
                            if 'grade ' in data.lower():
                                grade_index = data.lower().find('grade')
                                grade_val = grade_index + 6
                                scale = scale * (int(data[grade_val]))
                            new_values[iterator] = int(new_values[iterator]) + (1 * scale)
            iterator += 1
        if chemo_indicator:
            df.insert(col_index, col_name + '-' + key, new_values, allow_duplicates = False)
        else:
            df.insert(col_index, key, new_values, allow_duplicates = False)
        key_num += 1
    if chemo_indicator:
        df.insert(col_index, col_name+'-Secondary Regimen', chemo_secondary, allow_duplicates=False)
    df = df.drop(columns = col_name)
    return df

### Therapy Complications

In [ ]:
unique_tokens = get_unique_tokens('Complication During Neoadjuvant Treatment', ['; ', ', and', ', ', ',  '], df)
therapy_dict = {
    'Cardiovascular': ['nstemi', 'coronary vasospasm', 'vascular spasm of the fingers', 'thrombophlebitis', 'dvt', 'atrial fibrillation'],
    'Blood/Bone Marrow': ['thrombocytopenia', 'low platelets', 'induced thromboctopenia', 'anemia', 'low wbc', 'neutropenia', 'sepsis', 'cytopenia', 'necrosis', 'bacteremia'],
    'Constitutioinal Symptoms': ['nose and lip bleeding', 'suicidal ideations', 'anorexia', 'sirs', 'dehydration', 'failure to thrive', 'fatigue', 'cold intolerance', 'night sweats', 'cold sensitivity', 'hair loss', 'dizziness', 'nausea', 'vomitting', 'tongue and throat sensitivity', 'alopecia', 'allergic reaction', 'abdominal cramping'],
    'Dermatology': ['periananl skin', 'rash', 'hand foot syndrome', 'gout flare', 'hand and foot', 'palmar-plantarerythrodysesthesia', 'dermatitis', 'cellulitis', 'pruritis', 'skin toxicity', 'skin erythema', 'desquamation', 'skin erythema with desquamation', 'skin reaction', 'skin irrritation', 'perineal and vaginal skin irritation', 'skin changes', 'pruritus', 'skin desquamation', 'hand-foot syndrome', 'skin issues', 'pilonidal abscess'],
    'Reproductive': ['genital herpes', 'erectile dysfunction', 'menorrhagia', 'vagina'], 
    'Pulmonary': ['pneumothorax', 'pe', 'pulmonary hypertension', 'tracheal bronchitis'], 
    'Renal': ['ureter obstruction', 'acute kidney injury', 'aki'],
    'Pain': ['low backache', 'pain', 'perirectal burning', 'pneumonia', 'thrombophlebitis', 'gout', 'cramping'],
    'Neurology': ['numbness in hands', 'neuropathy', 'grade 1 taste alteration', 'dysuria', 'numb fingers', 'chemo brain'], 
    'Gastrointestinal': ['diarrhea', 'urinary freq', 'constipation', 'esophageal spasm', 'dysuria', 'urinary incontinence', 'fecal obstruction', 'gi bleed', 'colitis', 'bowel obstruction', 'incontinence', 'sbo', 'dysphagia', 'mucositis', 'crohn', 'pyroderma gangrenosum', 'hernia', 'ileitis', 'mouth sores', 'dysuria', 'mouth ulcers', 'enteritis', 'pneumatosis', 'diverticulitis', 'uti', 'urinary obstructive', 'mucousitis', 'stomatitis'],
    'Rectum': ['proctitis', 'perirectal burning', 'rectal pain' 'dermatitis and superimposed cellulitis of the periananl skin', 'anorectal pain', 'fistula', 'perineal and inguinal abscess', 'rectal bleeding', 'rectal irritation', 'rectal pain and bleeding', 'anal pruritis/irritation', 'perianal irritation', 'hemorrhoids', 'perirectal abscess', 'prostatitis']
}
df = bin('Complication During Neoadjuvant Treatment', therapy_dict, df)
df

### Operative Complications

In [ ]:
unique_vals = get_unique_tokens('Type of Intraoperative Complication ', [','], df)
operative_dict = {
    'GI Mesenteric Vessel Injury': ['mesenteric injury', 'colonic ischemia requiring excision of colon to hepatic flexure', 'ischemia noted without obvious vascular injury/disruption. distal transverse and descending colon were resected along with appendectomy', 'colorectal anastomosis', 'ischemia to colon requiring deloters procedure'],
    'GI Rectal Injury': ['bladder injury - Urinary tract hole in rectal stump after firing stapler', 'tear in rectum distal to anastomosis', 'entered the vagina and the rectal lumen accidentally', 'rectal injury', 'proctotomy'],
    'GI Anastomotic Leak': ['anastamotic defect', 'anastomotic leak requiring dli', 'staples did not hold rectal stump closed', 'anatosmotic leak repaired', 'positive leak test'],
    'Vaginal Injury': ['vaginal injury', 'hole created in posterior vaginal wall', 'entered the vagina and the rectal lumen accidentally', 'colpotomy'],
    'GI Small Bowel Injury': ['small bowel cautery injury', 'bowel injury', 'small bowel enterotomy'],
    'Prostate Injury': ['tear in rectum distal to anastomosis'],
    'Bleeding-Operative': ['bleeding', 'acidosis'],
    'Urinary Tract Injury': ['bladder injury', 'ureteral injury', 'coagulopathy - abd left open and taken to icu', 'urethral injury'],
    'Spleen Injury': ['splenic capsule tear']
}
df = bin('Type of Intraoperative Complication ', operative_dict, df)
df

### Organs Invaded
Need to determine relationship to Other Organ Involvement col

In [ ]:
organ_tokens = get_unique_tokens('Organs Invaded', [', ', ' and '], df)
organ_dict = {
    'Reproductive-Invaded': ['vagina', 'vas deferens', 'cervix', 'prostate', 'seminal vesticles', 'uterus', 'fallopian tube', 'ovaries', 'scrotum', 'ovary'],
    'Colon': ['colon'],
    'Anus': ['anus', 'peri-anal skin with metastatic lesion'],
    'Urinary Tract': ['urinary', 'periureteral soft tissue', 'bladdder', 'ureter', 'bladder'],
    'Nodes': ['nodes'],
    'Sacrum-Invaded': ['pre-sacral tissue', 'sacrum'],
    'Liver-Invaded': ['liver'],
    'Pelvic Bone': ['pelvic wall', 'obturator side wall', 'coccyx', 'pelvis', 'pelvic side wall', 'perisacral soft tissue'],
    'Pelvic Floor': ['pelvic floor muscle'],
    'Omentum': ['omentum'],
    'Spleen': ['spleen'],
    'Peritoneum': ['peritoneum'],
    'Soft Tissue': ['ab wall', 'buttocks'],
    'Small Bowel-Invaded': ['small bowel', 'ileum'],
    'Stomach': ['stomach'],
    'Mesentery': ['mesentary', 'mesentery'],
    'Vessels': ['iliac vessels']
}
df = bin('Organs Invaded', organ_dict, df)
df

### Chemo Regimens
Make a seperate feature that indicates if a patient had to switch regimens

In [ ]:
regimen_cols = ['Neoadjuvant Chemo Regimen', 'Adjuvant Chemo Regimen', 'Chemotherapy Regimen-Recurrence']
unique_regimens = []
for col in regimen_cols:
    regimen_tokens = get_unique_tokens(col, [', ', ' + ', '+', ' --> ', '--> ', ' -->', '-->', ' -> ', ' ->', ';', '; ', ' then ', '/', ', switched to ', ', and ', ' ,', ' and '], df)
    for token in regimen_tokens:
        unique_regimens.append(token)
tokens = []
for value in unique_regimens:
    trigger = False
    for token in tokens:
        if (value == token.lower()) or (value == 'no'):
            trigger = True
    if (not trigger):
        tokens.append(value)
    trigger = False
chemo_dict = {
    'Folfox': ['xelox', 'folfox', 'oxaliplatin', 'flox', 'capox', 'folfirinox', 'capeox', 'xeliri', 'foflox', 'xeleri'],
    '5FU': ['xeloda', 'capecitabine', '5-fu-based', '5-fu', 'leucovorin', '5fu', 'leocovorin', 'leukovorin', 'cabecitabine'],
    'Biologic': ['avastin', 'bbi', 'panitumumab', 'pembro', 'nivolumab', 'erbutux', 'cetuximab', 'erbitux', 'bev', 'cituximab', 'vectibix', 'xl888 trial', 'panitumaumab', 'lonsurf', 'ipilimumab', 'avastn', 'cetuximab', 'avastatin', 'avasrin'],
    'Folfiri': ['irinotecan', 'folfiri', 'capiri', 'bi5013', 'xeliri', 'folriri'],
    'Interleukin': ['nktr-214 (pegylated il-2)'],
    'Alt Chemo': ['etoposide', 'camtosar', 'cisplatin', 'carboplatin', 'carboxyplatin', 'mmc', 'other']
}
for i in regimen_cols:
    df = bin(i, chemo_dict, df, chemo_indicator=True)
df

## 5. Transform data
Some data in this dataset needs to be transformed to fit the same scale as other, related features.
### Columns to transform with units from/to in parenthesis (a * indicates that the column needs to be renamed): 
- Duration of Neoadjuvant Chemo-Days (Months/Days) *
- Duration of Neoadjuvant ChemoXRT-Days (Months/Days) *
- Adjuvant Chemo Duration-Days (Weeks/Days) *
- Adjuvant Chemoradiation Duration-Days (Weeks/Days) *

In [ ]:
#running this cell more than once WILL result in CORRUPTED VALUES, restart kernel before running

months_to_days = ['Duration of Neoadjuvant Chemo-Days', 'Duration of Neoadjuvant ChemoXRT-Days']
for feat in months_to_days:
    df[feat] = df[feat].infer_objects()
    df[feat] = df[feat].multiply(30)

weeks_to_days = ['Adjuvant Chemo Duration-Days', 'Adjuvant Chemoradiation Duration-Days']
for feat in weeks_to_days:
    df[feat] = df[feat].infer_objects()
    df[feat] = df[feat].multiply(7)

## 6. Drop single value columns
Just like it sounds, these are columns that only contain a single value. These will only serve to distract our model since it cannot draw an inference from a static value, so we need to drop them from our dataset.

In [ ]:
from feature_selector import FeatureSelector
fs = FeatureSelector(data = df)
fs.identify_single_unique()
print(fs.ops['single_unique'])
fs.plot_unique()

In [ ]:
df = df.drop(columns=fs.ops['single_unique'])
df

## 7. Clean dataframe dtypes
Transforming months to days in step 5 caused a lot of errors that needed to be fixed since a few samples had strings instead of integers which led to issues when multiplying. This means that we need to go back through the csv and identify any columns that have values with dtypes that are not uniform accross all samples. This will help moving forward when we have to deal with outliers and imputation. This really should have been one of the first steps, but as we work through this we are modifying the main dataset so it will apply to all cells.

### The column 'Distance from Sphincters' was misclassified as an Object but contains only floats, will have to cast it manually...
### 'Type of Anastomosis' contains integers/floats but is a categorical col...

In [ ]:
df = df.infer_objects()
df['Distance from Sphincters'] = pd.to_numeric(df['Distance from Sphincters'], errors = 'coerce')
df['Type of Anastomosis'] = df['Type of Anastomosis'].astype('category')

### Handling Distal Circumferential or Radial Margin (mm)
Split between 0 and >0. For convience, all '>0' values in the dataset have been changed to '1.0' to avoid an error.

In [ ]:
df['Distal Circumferential or Radial Margin (mm)'] = pd.cut(df['Distal Circumferential or Radial Margin (mm)'], [-1000000000, 0, 1000000000], labels = ['0', '>0'])
df['Distal Circumferential or Radial Margin (mm).1'] = pd.cut(df['Distal Circumferential or Radial Margin (mm).1'], [-1000000000, 0, 1000000000], labels = ['0', '>0'])

### Handling additional margins
- Radial margin should be grouped into >=3 
- Distal margin should be grouped into >=1

In [ ]:
df['Radial Margin Distance (mm)'] = pd.cut(df['Radial Margin Distance (mm)'], [-1000000000, 3, 1000000000],  right = False, labels = ['<3', '>=3'], include_lowest = True)
df['Radial Margin Distance (mm).1'] = pd.cut(df['Radial Margin Distance (mm).1'], [-1000000000, 3, 1000000000], right = False, labels = ['<3', '>=3'], include_lowest = True)

df['Distal Margin Distance (cm)'] = pd.cut(df['Distal Margin Distance (cm)'], [-1000000000, 1, 1000000000], right = False, labels = ['<1', '>=1'], include_lowest = True)
df['Distal Margin Distance (cm).1'] = pd.cut(df['Distal Margin Distance (cm).1'], [-1000000000, 1, 1000000000], right = False, labels = ['<1', '>=1'], include_lowest = True)

### Converting to categorical dtype
Now that all numeric values have been converted to numeric dtypes, the only thing we need to do now is to convert all remaining Objects to categorical. This will help us do outlier correction and imputation without going over the dataset by hand again.

In [ ]:
to_cat = df.select_dtypes(include = 'object')
for col in to_cat.columns:
    df[col] = df[col].astype('category')

In [ ]:
#we are first going to infer dtypes for all objects in pandas so we can see and drop the samples that keep pandas from inferring the proper dtypes
dtypes = df.dtypes
dtypes.to_csv('inferred_dtypes.csv')
df.to_csv('US_RCC_Dataset_dtype_cleaning.csv')

### Cleaning log:
Below is a list of modifications made to the main dataset. It is extensive, but necessary in case one or more introduces bias into the model and we need to undo it. In cases where samples were dropped if they contained a certain value or qualifier, no more than about 10 samples were dropped at a time. Keep in mind that undoing certain edits below may cause a sample to raise an error in another column/feature.
- Dropped samples with the suffixes 'months' and 'cycles' from Adjuvant Chemo and Chemoradiation Duration cols
- Dropped samples prefixed with '<' from the col CEA (ng/ml)
- Dropped the single samples with the value 'Carcinoma in stiu' from Pre-Treatment T-Stage by MRI and ERUS
- Fixed UTF-8 encoding error that represented the >= symbol with 'â‰¥', replaced with '>='
- Dropped samples with the value 'Tumor deposits in subserosa/mesentery/non-peritonealized perirectal tissues' from Pre-Treatment N-Stage
- Dropped samples with the value 'â‰¥10-19 yearsNot applicable years' from Crohn's Disease
- Dropped sample with the value '4' in Weight Loss
- Dropped samples with Database ID 'UP0272' and 'VU025', date of diagnosis was not a fully formated date and was causing errors
- Corrected misspelling 'Tumor invades through muscularis propia'
- Changed 'Tumor invades muscularis propria' to 'Tumor invades through muscularis propria' (UNDONE)
- Dropped samples with the value 'T0N0' from Pre-Treatment AJCC Staging
- Dropped the lone sample with the value '4-6 regional lymph nodes' in Pre-Treatment N-Stage by MRI
- Dropped all samples prefixed with '>' and 'greater than' from columns indicating the Number of Mesorectal/Retroperitoneal/Pelvic Nodes in the pre-treatment section
- Changed 'None' values to 'No' in col Ureteral Obstruction
- Dropped the two samples with 'Boost dose' values in Radiation Dose
- Dropped all samples prefixed with '>' in Number of Mesorectal/Retroperitoneal/Pelvic Nodes in the post-treatment section
- Dropped the single '>0.44' value from Post-Treatment Tumor Diameter (cm)
- Dropped the single '>1' value from # Lung Lesions on Imaging in Post-Treatment
- Changed the single 'Liver metastases' value to 'Synchronous liver metastases' in CURRENT Tumor Category
- Changed 'Total proctocolectomy' to 'APR' and dropped samples with 'Colostomy', 'Partial hepatectomy', and 'EMR' in Type of Operation of Rectal Tumor
- Changed 'No reason stated' values to 'Other' in Reason for Conversion to Open
- Dropped two samples with value 'â‰¥3' from Number of Staple Fires to Transect Rectum
- In col Portion of Sphincters Resected, Defect Closure, Peritoneal Perforation, Positive Margin Requiring Re-excision, Need for Radical Resection, Intervention Required, Ostomy Reversed after Resolution of Leak, and Radiation changed 0 to 'No' and 1 to 'Yes'
- Dropped samples with the value '>10' in # Liver Lesions in the Liver Resection section
- Dropped single sample with the value 'Carcinoma in situ' from Post-Treatment T-Stage by ERUS
- Dropped all samples with the value '4' in Tumor Differentiation
- Dropped all samples prefixed with '>' or '<' from Proximal Margin Distance (cm), Distal Margin Distance (cm), and Radial Margin Distance (mm)
- Changed '1' values to '1 regional lymph node' and fixed utf-8 encoding error for >= symbol in N-Stage.1
- Dropped single '1' value from Type of Operation of Rectal Tumor
- Dropped sample with Database ID 'UP0072', Anastonomic Leak diagnosis date was 7/11/01. Probably a typo for 2011, but better to drop just in case
- Dropped samples with the value '3' in Method of Leak Diagnosis
- Dropped samples with values '1', '2', and '3' in Type of Intervention
- Fixed encoding error in Time to Readmission and Time to Death, â‰¤ to <=
- Dropped single sample with the value 'UnkNown' in Adjuvant Chemoradiation
- Dropped samples with values of 1433.2 and 1436.1 in Recurrence Free Survival (Months)
- Dropped values not containing 'adenocarcinoma' from Histopathologic Type

## 8. Handling outliers
In numeric columns, outliers can signifigantly impare the function of a model. However, it can also be beneficial for the model to see that a value is higher than average. 
### Brainstorming
The first thing we need to do is filter out any non-numeric data, thanfully this will now be easy since we went through and cleaned out dtypes already. Ideally, I would like to create a folder containing data on each feature. I would like to use sklearn to find the 5-10 most related features and build graphs for each via seaborn's pairplot. Alternatively, we could simply create a large pairplot for each feature, acting as some form of a heat map.
#### Given that the methods to adjust outliers vary in end result given their application (regression or classification), it may be worth exploring implementing outlier adjustments in the dynamic learning process. Ideally, the method of fixing outliers will change based on whether the data is currently being used for regression or classification. This could also be an opprotunity to implement Minkowski error
### End Approach
Having read a few whitepapers that indicate the most effective methods of outlier detection and adjustment vary by whether or not the application is regression or classification. Since the proposed archetecture contains both, it would be un-ideal to implement just one for the ground truth dataset. Therefore we will focus more on the second proposal in the brainstorming section, implementing outlier correction on the fly during training. However, we will be creating two pairplots of features so anyone outside of this use case can visualize distributions and outliers of this cleaned version of the dataset for their own application.
#### Had to scrap pair grids, cell would not execute. Commented out code in case someone would like to borrow it
### Proposed pairplots
1. Box and Scatter plots 
2. Distribution and Bivariate kde plot

In [ ]:
import seaborn as sns; sns.set()
sns.set_palette("muted")
sns.set_color_codes()

In [ ]:
#box_scat = sns.PairGrid(df, dropna=True)
#box_scat = box_scat.map_diag(sns.boxplot, color = "o", orient = "v")
#box_scat = box_scat.map_offdiag(sns.scatter)
#box_scat.savefig("grid_boxandscat.png")

In [ ]:
#def hexabin(x, y):
#    plt.hexbin(x, y, gridsize=50, cmap="Blues")

In [ ]:
#distr_hex = sns.PairGrid(df, dropna=True)
#distr_hex = distr_hex.map_diag(sns.distplot, color = "o")
#distr_hex = distr_hex.map_offdiag(hexabin)
#istr_hex.savefig("grid_distrandhex.png")

## 9. Imputation
This dataset is incredibly dirty and has a lot of missing values, this means we have to impute, or fill in, these missing values. However the method of which we do this depends quite heavily on the specific feature and the data within, this means the dataset has to be gone over by hand. 
### Process:
The approach I will be taking here will be very similar to binning. The features will be split up into dictionaries to impute a defined default value, or a conditional value determined by a related feature. All remaining empty values will be filled by MICE, this is in lew of simply selecting the median or maximum occurred values from the column as such practices are less than ideal given they can be less accurate and even introduce bias into the data. MICE is done last since it works best with MAR data.
### Default values for categorical cols:
Typically a value such as 'None' would be imputed uniformly if a categorical column (with multiple categories, ie not simply yes and no) does not apply to a sample. However, this dataset uses many different NA values which means we need to locate these and replace them with a uniform alternative so these columns will be easier to work with when handling things like conditional logic.